In [1]:
#data are pulled from Google BigQuery using SQLimport pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [4]:
a = list(range(10))
a = [ '0' + str(i) for i in a]
dfs = [pd.read_table('event-0000000000'+i,sep = ',') for i in a]
df = pd.concat(dfs)
df.head()

,sqldate,quadclass,goldsteinscale,numarticles,avgtone
0,20180222,2,6.4,10,1.309329
1,20180221,1,-0.3,1,-2.417582
2,20180220,4,-9.2,2,-0.522388
3,20180220,4,-9.2,1,-0.615385
4,20180221,1,-0.1,2,-0.421348


In [2]:
df_mention = [pd.read_table('mentions'+str(j), sep=',') for j in range(10)]
df_mention = pd.concat(df_mention)
df_mention.head()

,sqldate,nummentions,numsources
0,20170902,18,1
1,20170830,9,2
2,20170831,16,8
3,20170901,14,7
4,20170901,15,1


In [5]:
df_mention = df_mention.loc[df_mention['sqldate']>20150000 ,]
df_mention.shape

(52271029, 3)

In [5]:
df = df.loc[df['sqldate']>20150000,]
df.shape

(52209023, 5)

In [3]:
df_mention.corr()

,sqldate,nummentions,numsources
sqldate,1.000000,-0.011905,0.000328
nummentions,-0.011905,1.000000,0.346716
numsources,0.000328,0.346716,1.000000


In [6]:
df_quad = pd.get_dummies(df['quadclass'])
#df_quad = df[['sqldate']].join(df_quad)
df_quad.shape
df_quad.head()

,1,2,3,4
0,0,1,0,0
1,1,0,0,0
2,0,0,0,1
3,0,0,0,1
4,1,0,0,0


In [24]:
#df = df[['sqldate','goldsteinscale','numarticles','avgtone']].join(df_quad)

In [25]:
df.dtypes

sqldate             int64
quadclass           int64
goldsteinscale    float64
numarticles         int64
avgtone           float64
dtype: object

In [7]:
b = pd.unique(df['sqldate'])

In [8]:
b.sort()

In [9]:
col1 = ['goldsteinscale', 'numarticles', 'avgtone']

In [29]:
#df_1 = [pd.DataFrame([sum([k for k in df[j][df['sqldate'] == i]]) for j in col1]) for i in b]

In [10]:
data = []

data.append([np.mean(df['goldsteinscale'][df['sqldate']==i]) for i in b])

In [11]:
data.append([np.mean(df['numarticles'][df['sqldate']==i]) for i in b])

In [12]:
data.append([np.mean(df['avgtone'][df['sqldate']==i]) for i in b])

In [13]:
df_1 = pd.DataFrame(np.array(data).transpose(),columns = ['goldstein','numarticles','avgtone'], index = b)

In [14]:
#df_articles = pd.DataFrame(np.array([[i for i in b], [np.mean(df['numarticles'][df['sqldate']==i]) for i in b]]),
#                           columns = ['date','numarticles'])
df_1.head()                       

,goldstein,numarticles,avgtone
20150101,1.897037,4.725926,-0.825348
20150102,1.292029,4.144928,-2.010586
20150103,0.500662,4.072848,-1.579904
20150104,0.078648,4.177936,-2.290629
20150105,-1.140491,4.365031,-2.789627


In [15]:
# df_avgtone = pd.DataFrame(np.array([[i for i in b], [np.mean(df['avgtone'][df['sqldate']==i]) for i in b]]),
#                           columns = ['date','avgtone'])

In [17]:
data_mention = []
data_mention.append([np.mean(df_mention['nummentions'][df_mention['sqldate']==i]) for i in b])

In [18]:
data_mention.append([np.mean(df_mention['numsources'][df_mention['sqldate']==i]) for i in b])

In [19]:
df_3 = pd.DataFrame(np.array(data_mention).transpose(),columns = ['avg mention','avg source'], index = b)

In [20]:
data_quad = []

data_quad.append([np.mean(df_quad[1][df['sqldate']==i]) for i in b])

In [21]:
data_quad.append([np.mean(df_quad[2][df['sqldate']==i]) for i in b])

In [22]:
data_quad.append([np.mean(df_quad[3][df['sqldate']==i]) for i in b])

In [23]:
data_quad.append([np.mean(df_quad[4][df['sqldate']==i]) for i in b])

In [24]:
df_2 = pd.DataFrame(np.array(data_quad).transpose(),columns = ['quad_1','quad_2','quad_3','quad_4'], index = b)

In [25]:
df_2.head()

,quad_1,quad_2,quad_3,quad_4
20150101,0.644444,0.148148,0.133333,0.074074
20150102,0.463768,0.282609,0.137681,0.115942
20150103,0.629139,0.092715,0.112583,0.165563
20150104,0.491103,0.177936,0.103203,0.227758
20150105,0.435583,0.138037,0.101227,0.325153


In [26]:
full_df = df_1.join(df_2).join(df_3)

In [28]:
full_df.head()

,goldstein,numarticles,avgtone,quad_1,quad_2,quad_3,quad_4,avg mention,avg source
20150101,1.897037,4.725926,-0.825348,0.644444,0.148148,0.133333,0.074074,4.800000,1.051852
20150102,1.292029,4.144928,-2.010586,0.463768,0.282609,0.137681,0.115942,4.144928,1.000000
20150103,0.500662,4.072848,-1.579904,0.629139,0.092715,0.112583,0.165563,4.178808,1.000000
20150104,0.078648,4.177936,-2.290629,0.491103,0.177936,0.103203,0.227758,4.249110,1.028470
20150105,-1.140491,4.365031,-2.789627,0.435583,0.138037,0.101227,0.325153,4.407975,1.033742


In [27]:
full_df.to_csv('FULL_DF2.csv')

In [29]:
full_df.corr()

,goldstein,numarticles,avgtone,quad_1,quad_2,quad_3,quad_4,avg mention,avg source
goldstein,1.000000,-0.005574,0.603847,0.630025,0.274719,-0.319499,-0.846469,-0.031480,-0.013271
numarticles,-0.005574,1.000000,0.057282,0.022788,0.077431,-0.232496,0.064739,0.958819,0.513763
avgtone,0.603847,0.057282,1.000000,0.559032,-0.042051,-0.209194,-0.602006,0.029398,-0.076970
quad_1,0.630025,0.022788,0.559032,1.000000,-0.475724,-0.416964,-0.773689,0.042861,-0.013310
quad_2,0.274719,0.077431,-0.042051,-0.475724,1.000000,-0.147542,0.057436,0.042111,0.027840
quad_3,-0.319499,-0.232496,-0.209194,-0.416964,-0.147542,1.000000,0.030434,-0.240225,-0.081000
quad_4,-0.846469,0.064739,-0.602006,-0.773689,0.057436,0.030434,1.000000,0.066539,0.051047
avg mention,-0.031480,0.958819,0.029398,0.042861,0.042111,-0.240225,0.066539,1.000000,0.487723
avg source,-0.013271,0.513763,-0.076970,-0.013310,0.027840,-0.081000,0.051047,0.487723,1.000000
